In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('C:/Users/bhatt/OneDrive/Desktop/whisper-main/ANN_Data/data1.csv', header=None)
print(data)
labels=pd.read_csv('C:/Users/bhatt/OneDrive/Desktop/whisper-main/ANN_Data/label1.csv',header=None)
print(labels)

             0         1         2         3         4         5         6   \
0     -0.446891 -0.013397  0.232645  2.156649  1.652923 -0.210531 -0.662227   
1      0.243747 -1.144175 -0.622214 -0.661979 -0.373315  0.520313 -1.307954   
2     -1.417799 -0.088833 -0.647181 -1.141497 -1.321100  0.686798 -1.045569   
3      0.656784  0.381201 -1.039011  1.285315  0.808324  1.428519 -1.144367   
4      1.098520  1.706769 -1.030370  2.001009  2.751551  1.202229 -0.941066   
...         ...       ...       ...       ...       ...       ...       ...   
19995  0.146324 -0.618555  0.547413  0.851301  0.838880 -1.248402  0.698199   
19996 -2.227923 -0.327196  1.258849 -0.945254  3.925262  0.316686 -1.061159   
19997 -0.800405  1.477997  0.090057 -0.973997 -0.204970  0.224539  0.302265   
19998 -1.217344 -0.785401 -0.413781 -0.335324  0.181671 -1.685733 -0.437981   
19999  1.601005 -0.890398 -1.425049  0.887501  0.167205 -0.395042 -0.512554   

             7         8         9         10      

In [3]:
data=data.values
npLabels=labels.values
print(data.shape)
print(npLabels.shape)

(20000, 20)
(20000, 1)
(20000, 20)
(20000, 1)


In [4]:
print(np.max(npLabels))
print(np.min(npLabels))

4
0
4
0


In [5]:
np.unique(npLabels)

array([0, 1, 2, 3, 4], dtype=int64)

array([0, 1, 2, 3, 4], dtype=int64)

In [6]:
num=np.max(npLabels)+1
oneHot=np.eye(num)[npLabels]
print(oneHot.ndim)
print(oneHot.shape)

3
(20000, 1, 5)
3
(20000, 1, 5)


In [7]:
oneHotRe=oneHot.reshape(20000, 5)
print(oneHotRe)

[[0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]]
[[0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]]


In [8]:
trainData,testDataDash=train_test_split(data,train_size=0.8,test_size=0.2,shuffle=False)
trainLabel,testLabelDash=train_test_split(oneHotRe,train_size=0.8,test_size=0.2,shuffle=False)

In [9]:
validData,testData=train_test_split(testDataDash,train_size=0.5,test_size=0.5,shuffle=False)
validLabel,testLabel=train_test_split(testLabelDash,train_size=0.5,test_size=0.5,shuffle=False)

In [10]:
h = 15

In [11]:
b1=np.zeros(h)
b2=np.zeros(5)
print(b1)
print(b2)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]


In [12]:
w1=np.random.normal(0, 1, (20,h))
w2=np.random.normal(0, 1, (h,5))

In [13]:
def actFunc(data,choice):

    if(choice==1):
        return np.tanh(data)
    elif(choice==2):
        numerator=np.exp(data)
        return numerator/np.sum(numerator,axis=1,keepdims=True)

In [14]:
alpha = 0.003

In [15]:
batch=1000

In [16]:
def costFunc(y,t):
    return -(t*(np.log(y))+(1-t)*np.log(1-y))

In [17]:
def forwFunc(data):
    a0=data
    z1=np.dot(a0,w1)+b1
    a1=actFunc(z1,1)
    z2=np.dot(a1,w2)+b2
    a2=actFunc(z2,2)
    return a0,z1,a1,z2,a2

In [18]:
def Acc(y,t,size):
    maxData=np.argmax(y,axis=1)
    maxLabel=np.argmax(t,axis=1)
    compare=np.equal(maxData,maxLabel)
    count=np.sum(compare)
    return (count/size)*100

In [19]:
epochs=500

In [20]:
for epoch in range(epochs):
    for iteration in range(int(len(trainData)/batch)):
        a0,z1,a1,z2,a2=forwFunc(trainData[iteration*batch:(iteration+1)*batch,:])
        y=a2
        labelBatch=trainLabel[iteration*batch:(iteration+1)*batch,:]
        del2=(y-labelBatch)
        del1=np.dot(del2,w2.T)*(1 - a1**2)
        dcdw2=np.dot(a1.T,del2)
        dcdw1=np.dot(a0.T,del1)
        dcdb1=np.sum(del1,axis=0)
        dcdb2=np.sum(del2,axis=0)
        w1=w1-alpha*dcdw1
        w2=w2-alpha*dcdw2
        b2=b2-alpha*dcdb2
        b1=b1-alpha*dcdb1
    a0,z1,a1,z2,a2=forwFunc(trainData)
    print("Training Cost",(np.sum(costFunc(a2,trainLabel)))/16000.0)
    print("Training accuracy",Acc(a2,trainLabel,len(trainLabel)))
    
va0,vz1,va1,vz2,va2=forwFunc(validData)
vOutput=va2
accuracy = Acc(vOutput,validLabel,2000.0)
print("validation accuracy")
print(accuracy)

Training Cost 1.4119220439843931
Training accuracy 68.86875
Training Cost 1.2308233546631235
Training accuracy 73.6625
Training Cost 1.4119220439843931
Training accuracy 68.86875
Training Cost 1.2308233546631235
Training accuracy 73.6625
Training Cost 1.120950376032111
Training accuracy 76.3875
Training Cost 1.013448143153476
Training accuracy 79.04375
Training Cost 0.9338755762144504
Training accuracy 80.84375
Training Cost 0.8815391839380816
Training accuracy 82.28750000000001
Training Cost 0.8423042659232114
Training accuracy 83.26875
Training Cost 0.8031876437968242
Training accuracy 84.30625
Training Cost 0.7643627731034763
Training accuracy 85.2375
Training Cost 1.120950376032111
Training accuracy 76.3875
Training Cost 1.013448143153476
Training accuracy 79.04375
Training Cost 0.9338755762144504
Training accuracy 80.84375
Training Cost 0.8815391839380816
Training accuracy 82.28750000000001
Training Cost 0.8423042659232114
Training accuracy 83.26875
Training Cost 0.803187643796824

Training Cost 0.5221986985383539
Training accuracy 90.41875
Training Cost 0.5209833321906487
Training accuracy 90.5
Training Cost 0.5197725388288386
Training accuracy 90.55
Training Cost 0.5185583711890661
Training accuracy 90.55
Training Cost 0.5173299462818626
Training accuracy 90.55
Training Cost 0.5160734095284737
Training accuracy 90.56875
Training Cost 0.514767902608739
Training accuracy 90.64375
Training Cost 0.513380890501228
Training accuracy 90.69375
Training Cost 0.5118965480264256
Training accuracy 90.68125
Training accuracy 90.55
Training Cost 0.5160734095284737
Training accuracy 90.56875
Training Cost 0.514767902608739
Training accuracy 90.64375
Training Cost 0.513380890501228
Training accuracy 90.69375
Training Cost 0.5118965480264256
Training accuracy 90.68125
Training Cost 0.5103692806979706
Training accuracy 90.725
Training Cost 0.5088893408965263
Training accuracy 90.77499999999999
Training Cost 0.5075153788667544
Training accuracy 90.77499999999999
Training Cost 0.5

Training Cost 0.47843887897369597
Training accuracy 91.4375
Training Cost 0.47828704862848714
Training accuracy 91.475
Training Cost 0.4781329563133668
Training accuracy 91.475
Training Cost 0.4779779186997235
Training accuracy 91.46875
Training Cost 0.4778233306992409
Training accuracy 91.46875
Training Cost 0.47843887897369597
Training accuracy 91.4375
Training Cost 0.47828704862848714
Training accuracy 91.475
Training Cost 0.4781329563133668
Training accuracy 91.475
Training Cost 0.4779779186997235
Training accuracy 91.46875
Training Cost 0.4778233306992409
Training accuracy 91.46875
Training Cost 0.4776703837769048
Training accuracy 91.46875
Training Cost 0.4775199498907695
Training accuracy 91.50625
Training Cost 0.47737254552885167
Training accuracy 91.53125
Training Cost 0.47722824093039
Training accuracy 91.53750000000001
Training Cost 0.4776703837769048
Training accuracy 91.46875
Training Cost 0.4775199498907695
Training accuracy 91.50625
Training Cost 0.47737254552885167
Trai

Training Cost 0.46840273218111045
Training accuracy 91.58125
Training Cost 0.4683156996729441
Training accuracy 91.57499999999999
Training Cost 0.46822842825341515
Training accuracy 91.56875
Training Cost 0.46814263474877
Training accuracy 91.56875
Training Cost 0.46840273218111045
Training accuracy 91.58125
Training Cost 0.4683156996729441
Training accuracy 91.57499999999999
Training Cost 0.46822842825341515
Training accuracy 91.56875
Training Cost 0.46814263474877
Training accuracy 91.56875
Training Cost 0.46806030695987216
Training accuracy 91.55624999999999
Training Cost 0.4679833305089958
Training accuracy 91.5625
Training Cost 0.467913108165423
Training accuracy 91.55
Training Cost 0.4678502971978792
Training accuracy 91.55
Training Cost 0.46806030695987216
Training accuracy 91.55624999999999
Training Cost 0.4679833305089958
Training accuracy 91.5625
Training Cost 0.467913108165423
Training accuracy 91.55
Training Cost 0.4678502971978792
Training accuracy 91.55
Training Cost 0.46

Training Cost 0.46231251991920524
Training accuracy 91.61875
Training Cost 0.46222377004144066
Training accuracy 91.60625
Training Cost 0.4621319207207332
Training accuracy 91.60625
Training Cost 0.4620365409234935
Training accuracy 91.61875
Training Cost 0.46231251991920524
Training accuracy 91.61875
Training Cost 0.46222377004144066
Training accuracy 91.60625
Training Cost 0.4621319207207332
Training accuracy 91.60625
Training Cost 0.4620365409234935
Training accuracy 91.61875
Training Cost 0.46193732594669734
Training accuracy 91.6125
Training Cost 0.46183416094271945
Training accuracy 91.625
Training Cost 0.4617271806219986
Training accuracy 91.6125
Training Cost 0.4616168135067748
Training accuracy 91.61875
Training Cost 0.4615037972302929
Training accuracy 91.63125
Training Cost 0.46193732594669734
Training accuracy 91.6125
Training Cost 0.46183416094271945
Training accuracy 91.625
Training Cost 0.4617271806219986
Training accuracy 91.6125
Training Cost 0.4616168135067748
Trainin

Training Cost 0.46261132351260187
Training accuracy 91.68125
Training Cost 0.46261290890680407
Training accuracy 91.68125
Training Cost 0.4626123378978633
Training accuracy 91.675
Training Cost 0.4626095982554038
Training accuracy 91.68125
Training Cost 0.46260469956156647
Training accuracy 91.68125
Training Cost 0.462597676462851
Training accuracy 91.6875
Training Cost 0.4625885898077157
Training accuracy 91.66250000000001
Training Cost 0.46257752579884753
Training accuracy 91.65625
Training Cost 0.46256459347223194
Training accuracy 91.66250000000001
Training Cost 0.4625885898077157
Training accuracy 91.66250000000001
Training Cost 0.46257752579884753
Training accuracy 91.65625
Training Cost 0.46256459347223194
Training accuracy 91.66250000000001
Training Cost 0.4625499209322608
Training accuracy 91.66250000000001
Training Cost 0.4625336508177346
Training accuracy 91.66250000000001
Training Cost 0.4625159354559238
Training accuracy 91.675
Training Cost 0.46249693209765824
Training ac

Training accuracy 91.74375
Training Cost 0.46121889064600735
Training accuracy 91.75
Training Cost 0.46119688116032764
Training accuracy 91.75
Training Cost 0.4611748811969667
Training accuracy 91.75625
Training Cost 0.4611529001908723
Training accuracy 91.75
Training Cost 0.4611309473909849
Training accuracy 91.75
Training Cost 0.46110903166416123
Training accuracy 91.75
Training Cost 0.46108716131626043
Training accuracy 91.75
Training Cost 0.46106534393592336
Training accuracy 91.75
Training Cost 0.461043586265391
Training accuracy 91.75
Training Cost 0.46102189410133426
Training accuracy 91.75
Training Cost 0.461000272227195
Training accuracy 91.75625
Training Cost 0.4609787243770891
Training accuracy 91.78125
Training Cost 0.4609572532299866
Training accuracy 91.78125
Training Cost 0.46093586043174695
Training accuracy 91.7875
Training Cost 0.4609145466417006
Training accuracy 91.8
Training Cost 0.46089331159986
Training accuracy 91.8
Training Cost 0.4608721542105185
Training accu

Training Cost 0.4598403412527393
Training accuracy 91.80625
Training Cost 0.45982632248595634
Training accuracy 91.81875000000001
Training Cost 0.4598129920601672
Training accuracy 91.81875000000001
Training Cost 0.4598004101022705
Training accuracy 91.83125
Training Cost 0.4597886402326459
Training accuracy 91.825
Training Cost 0.4597777498477364
Training accuracy 91.83125
Training Cost 0.4597678104859855
Training accuracy 91.825
Training Cost 0.4597588982720393
Training accuracy 91.825
Training Cost 0.4597510944204773
Training accuracy 91.825
Training Cost 0.45974448575986915
Training accuracy 91.825
Training Cost 0.4597391652063109
Training accuracy 91.83125
Training Cost 0.4597352320650244
Training accuracy 91.81875000000001
Training Cost 0.45973279195549344
Training accuracy 91.8125
Training Cost 0.4597319560156261
Training accuracy 91.80625
Training Cost 0.45973283879978705
Training accuracy 91.8
Training Cost 0.4597355538636108
Training accuracy 91.7875
Training Cost 0.459740205